# Fileupload

In [85]:
import pandas as pd
#import os
import numpy as np
import statsmodels.api as sm
import scipy
import plotly.express as px

In [40]:
 os.getcwd()

'/home/wseis/qmra/tools'

In [41]:
data = pd.read_csv("./logremoval/testdata.csv")

491438.3076923077

In [73]:
Xin = np.ones_like(data["inflow"])
Xout = np.ones_like(data["inflow"])
nb_inflow = sm.NegativeBinomial(data["inflow"], Xin).fit()
nb_outflow = sm.NegativeBinomial(data["outflow"], Xout).fit()

Optimization terminated successfully.
         Current function value: 13.853863
         Iterations: 6
         Function evaluations: 7
         Gradient evaluations: 7
Optimization terminated successfully.
         Current function value: 4.784866
         Iterations: 4
         Function evaluations: 7
         Gradient evaluations: 7


In [74]:
def simulate_negbin(model):
    mu = np.exp(model.params[0])
    p = 1/(1+np.exp(model.params[0])*model.params[1])
    n = np.exp(model.params[0])*p/(1-p)
    return np.random.negative_binomial(n, p, 10000)
    

In [116]:
inflow = simulate_negbin(nb_inflow)
outflow = simulate_negbin(nb_outflow)
df = pd.DataFrame({"inflow": inflow+1, "outflow": outflow+1})

In [117]:
df["outflow"] = df["outflow"]+1
df["inflow"] = df["inflow"]+1


In [121]:
df["login"] = np.log10(df["inflow"])

In [119]:
df["LRV"] =np.log10((df["inflow"])/df["outflow"])


In [131]:
df.describe()

inflow     6327.800000
outflow       1.000000
q             2.199167
login         3.801253
Name: 0.1, dtype: float64

In [144]:
df.quantile([.1, .5, .9])

,inflow,outflow,q,login
0.1,6327.8,1.0,2.199167,3.801253
0.5,211670.5,25.0,3.876889,5.325660
0.9,1339456.0,176.0,5.349366,6.126928


In [143]:
np.quantile(df["inflow"], 0.1)

6327.8